In [37]:
import pandas as pd
import time
import openai
import sys
import tiktoken
import numpy as np
import warnings
import io
import toml
import threading
import random
import logging 
import streamlit as st
from dotenv import load_dotenv
from htmlTemplates import css, bot_template, user_template
from PyPDF2 import PdfReader
#from langchain.text_splitter import CharacterTextSplitter
from preprocess import *
from embeddings import *
from main_functions import *
from docx import Document

In [2]:
secrets ="secrets.toml"

#secrets
toml_dir = secrets

with open(toml_dir, 'r') as f:
    config = toml.load(f)

# Set up OpenAI API credentials
# azure required env vars commented out 
openai.api_type = config['api_type']
openai.api_base = config['api_base']
openai.api_version = config['api_version']
openai.api_key = config['api_key']
# Set up engine name
engine_name = config['engine_name']

#Completion on knowlede store config
EMBEDDING_MODEL = config['EMBEDDING_MODEL']

In [3]:
def define_dataframe_types(df):
    df.loc[:, 'heading'] = df['heading'].astype(str)
    df.loc[:, 'title'] = df['title'].astype(str)
    df.loc[:, 'tokens'] = df['tokens'].astype(int)
    df.loc[:, 'content'] = df['content'].astype(str)

In [4]:
def reset_messages_ai():
    global messages_ai
    # Reset messages_ai to its default state or an empty list/dictionary, depending on your implementation
    messages_ai = []

In [78]:
def load_dfs(local_toml, idx_load = ['title','heading']):
    global data_sources
    global doc_embeddings 
    global code_embeddings 
    global df_code
    global df_doc
    global app_prompt_list
 
    df = pd.read_parquet(config[local_toml])

    df_code= df
    define_dataframe_types(df_code)

    code_embeddings = load_embeddings(df_code)
    df_code = df_code.set_index(idx_load)
    reset_messages_ai()

In [51]:
def convert_text_token():
    pdf_folder = 'data_pdf'
    output_csv = 'output1_new.csv'

    process_folder(pdf_folder, output_csv)

In [62]:
df_data = pd.read_csv('output1_new.csv')

df_data = df_data[df_data['title']=='leaflet']

# Apply embeddings to each content cell and store the list of embeddings
df_data['embeddings'] = df_data['content'].apply(get_embeddings)
df_data

,title,heading,content,tokens,embeddings
114,leaflet,Page 1,53 B. PACKAGE LEAFLET,7,"[-0.0006750855827704072, 0.004950627684593201,..."
115,leaflet,Page 2,54 Package leaflet: Information for the patien...,598,"[-0.01264230627566576, 0.013442616909742355, 0..."
116,leaflet,Page 3,55 Alopecia areata Olumiant is used to treat a...,742,"[-0.027777131646871567, -0.005803436506539583,..."
117,leaflet,Page 4,56 coagulation defect is identified in you or ...,675,"[-0.013979101553559303, -0.0021750815212726593..."
118,leaflet,Page 5,57 Pregnancy and breast-feeding If you are pre...,623,"[1.0560002010606695e-05, 0.009587787091732025,..."
119,leaflet,Page 6,58 You can take the tablets either with or wit...,565,"[0.002330534625798464, 0.0032498009968549013, ..."
120,leaflet,Page 7,59 Uncommon (may affect up to 1 in 100 people)...,573,"[0.0008120652055367827, -0.004402050748467445,..."
121,leaflet,Page 8,60 What Olumiant looks like and contents of th...,785,"[-0.0002589018549770117, 0.008265532553195953,..."
122,leaflet,Page 9,61 France Lilly France Tél: +33-(0) 1 55 49 3...,557,"[-0.010335579514503479, 0.005636405665427446, ..."
123,leaflet,Page 10,62 Your name: ________________________________...,323,"[-0.008183985948562622, -0.024053573608398438,..."


In [52]:
convert_text_token()

In [5]:
def convert_token_embeddings():
    # Replace 'your_input.csv' with the path to your CSV file
    input_csv = 'output1_new.csv'
    process_dataframe(input_csv)

In [6]:
convert_token_embeddings()

ewkljfehffuhsufhgweufhwugweiyg
114    leaflet
115    leaflet
116    leaflet
117    leaflet
118    leaflet
119    leaflet
120    leaflet
121    leaflet
122    leaflet
123    leaflet
Name: title, dtype: object
Output saved as output1_new_with_embeddings.csv and output1_new_with_embeddings.parquet


In [9]:
def respond(message, chat_history):
    
      
    global messages_ai
    global file_name_chat
    global chat_id
    print("Message_AI: ",messages_ai)

    bot_message, messages_ai = initiate_countdown(basic_chain_parallel_single, messages_ai, message,code_embeddings, df_code, seconds = 60)
    #write_messages_ai_to_txt(messages_ai)
    chat_history.append((message, bot_message))  
    return messages_ai, chat_history

In [15]:
load_dfs("medical_data")

In [16]:
message = "hi"
chat_history = []
messages_ai, chat_history = respond(message, chat_history)

Message_AI:  []


In [22]:
messages_ai

[{'role': 'user', 'content': 'hi'},
 {'role': 'assistant',
  'content': "I'm sorry, I can only assist with Lilly medicine related queries."}]

In [18]:
chat_history

[('hi', "I'm sorry, I can only assist with Lilly medicine related queries.")]

In [21]:
for i, message in enumerate(messages_ai):
    if i % 2 == 0:
        print(i)
        print(message['role'])
        print(message['content'])
    else:
        print(i)
        print(message['role'])
        print(message['content'])
        

0
user
hi
1
assistant
I'm sorry, I can only assist with Lilly medicine related queries.


In [20]:
RESPONSE = [{'role': 'user', 'content': 'hi'}, {'role': 'assistant', 'content': "I'm sorry, I can only assist with Omuliant related queries."}, {'role': 'user', 'content': 'what is olumiant?'}, {'role': 'assistant', 'content': 'Omuliant is a medication primarily used for the treatment of certain diseases such as Rheumatoid arthritis and Juvenile idiopathic arthritis. It contains the active ingredient baricitinib, which helps to reduce inflammation and relieve symptoms associated with these conditions. \n\nPlease note that this response is based on the provided context and my knowledge base. If you require more specific information or have any other questions related to Omuliant, feel free to ask.'}]

In [34]:
for i, message in enumerate(RESPONSE):
    if i % 3 == 0:
        print(i)
        print(message['role'])
        print(message['content'])
    else:
        print(i)
        print(message['role'])
        print(message['content'])

0
user
hi
1
assistant
I'm sorry, I can only assist with Omuliant related queries.
2
user
what is olumiant?
3
assistant
Omuliant is a medication primarily used for the treatment of certain diseases such as Rheumatoid arthritis and Juvenile idiopathic arthritis. It contains the active ingredient baricitinib, which helps to reduce inflammation and relieve symptoms associated with these conditions. 

Please note that this response is based on the provided context and my knowledge base. If you require more specific information or have any other questions related to Omuliant, feel free to ask.


## Splitting of the pdf

In [35]:
pip install PyPDF2



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import PyPDF2

def split_pdf(input_pdf, keyword):
    pdf_reader = PyPDF2.PdfReader(input_pdf)
    num_pages = len(pdf_reader.pages)
    print(num_pages)
    # Find the keyword page
    keyword_page = None
    for page_num in range(num_pages):
        page_text = pdf_reader.pages[page_num].extract_text()
        
        if keyword in page_text:
            keyword_page = page_num
            break

    if keyword_page is None:
        print("Keyword not found in the PDF.")
        return

    # Split the PDF
    pdf_writer1 = PyPDF2.PdfWriter()
    pdf_writer2 = PyPDF2.PdfWriter()

    for page_num in range(num_pages):
        if page_num < keyword_page:
            pdf_writer1.add_page(pdf_reader.pages[page_num])
        else:
            pdf_writer2.add_page(pdf_reader.pages[page_num])

    with open("part1.pdf", "wb") as output1:
        pdf_writer1.write(output1)

    with open("part2.pdf", "wb") as output2:
        pdf_writer2.write(output2)

    print("PDF split successfully.")

# Usage example
split_pdf("data_pdf/Olumiant.pdf", "B. PACKAGE LEAFLET")


62
PDF split successfully.


In [26]:
import ast
file_path = "messages_ai_debug.txt"
with open(file_path, 'r') as file:
    # Read the entire contents of the file
    text_data = file.read()
    list_object = ast.literal_eval(text_data)
    

In [29]:
list_object[0]['content']

'serious side effects (shingles and lung infection)'

In [25]:
text_data

'[\n    {\n        "role": "user",\n        "content": "serious side effects (shingles and lung infection)"\n    },\n    {\n        "role": "assistant",\n        "content": "The serious side effects of OLUMIANT (baricitinib) can include infections such as shingles and lung infections. It is important to note that these serious infections can occur while taking OLUMIANT. If you experience symptoms such as fever, chills, cough, or any signs of infection, it is crucial to contact your healthcare provider immediately. They will be able to evaluate your symptoms and provide appropriate medical guidance. It is also important to discuss any concerns or questions about the potential side effects of OLUMIANT with your healthcare provider before starting the medication.\\n\\nReference for answer: OLUMIANT Prescribing Information, page 5."\n    },\n    {\n        "role": "user",\n        "content": "what are the possible side effects of olumiant\\u00ae (baricitinib)?"\n    },\n    {\n        "rol

In [34]:
l = []
for d in list_object:
    l.append(d['content'])

In [35]:
l

['serious side effects (shingles and lung infection)',
 'The serious side effects of OLUMIANT (baricitinib) can include infections such as shingles and lung infections. It is important to note that these serious infections can occur while taking OLUMIANT. If you experience symptoms such as fever, chills, cough, or any signs of infection, it is crucial to contact your healthcare provider immediately. They will be able to evaluate your symptoms and provide appropriate medical guidance. It is also important to discuss any concerns or questions about the potential side effects of OLUMIANT with your healthcare provider before starting the medication.\n\nReference for answer: OLUMIANT Prescribing Information, page 5.',
 'what are the possible side effects of olumiant® (baricitinib)?',
 'The possible side effects of OLUMIANT (baricitinib) can vary from person to person. Common side effects may include upper respiratory tract infections, nausea, and changes in blood test results (such as incre

In [38]:
doc = Document()

    # Add each item from the list to the document as a paragraph
for item in l:
    doc.add_paragraph(item)

    # Save the document
doc.save("bot_output.docx")

In [57]:
import json
cwd = os.getcwd()
medicine_path = os.path.join(cwd, "prompt_config.json")
with open(medicine_path, 'r') as v:
    medicines = json.load(v)
medicines

[{'medicine': 'OLUMIANT',
  'code_header': "You are an expert medical AI assistant with up-to-date knowledge for OLUMIANT, a medication developed by Eli Lilly. \n                OLUMIANT is a medication featuring the active ingredient baricitinib, primarily used to treat rheumatoid arthritis. \n                You excel in providing expert advice on this drug based on user inquiries. Your responses are rooted in integrity, excellence, \n                and respect for individuals, ensuring accuracy and reliability in all information shared. \n                Answer the user's query using only this Referenced information.                *** Reference information ***    \n                ",
  'code_answering_instructions': "\n[IMPORTANT INSTRUCTIONS]:\n- If a query does not pertain to legitimate OLUMIANT related inquiry, respond with: 'I am sorry, I can only assist with OLUMIANT related queries.'\n- If the necessary information for answering 'Q:' is not in the provided 'Referenced inform

In [64]:
medicine = "MOZZARO"

In [58]:
if medicines[0]['medicine'] == medicine:
    code_header = medicines[0]['code_header']
    code_answering_instructions = medicines[0]['code_answering_instructions']
code_header

"You are an expert medical AI assistant with up-to-date knowledge for OLUMIANT, a medication developed by Eli Lilly. \n                OLUMIANT is a medication featuring the active ingredient baricitinib, primarily used to treat rheumatoid arthritis. \n                You excel in providing expert advice on this drug based on user inquiries. Your responses are rooted in integrity, excellence, \n                and respect for individuals, ensuring accuracy and reliability in all information shared. \n                Answer the user's query using only this Referenced information.                *** Reference information ***    \n                "

In [59]:
code_answering_instructions

"\n[IMPORTANT INSTRUCTIONS]:\n- If a query does not pertain to legitimate OLUMIANT related inquiry, respond with: 'I am sorry, I can only assist with OLUMIANT related queries.'\n- If the necessary information for answering 'Q:' is not in the provided 'Referenced information', say: 'Sorry, my knowledge base lacks the relevant information to address your question.'\n- Upon finding the required information in the 'Referenced information', address the query or complete the task ensuring you also follow the next point.\n- Conclude all responses prior to providing reference source by either:\n  1. Offering further assistance\n  2. Providing options for further exploration\n- Give detailed comprehensive answers, using all available useful information.\n- Reference the source of your answer at the end, using the format: 'Reference for answer: <Document Name>, <Page Number>.'\n\n\n Input Q:\n"

In [65]:
code_header = f"""You are an expert medical AI assistant with up-to-date knowledge for {medicine}, a medication developed by Eli Lilly. 
                {medicine} is a medication featuring the active ingredient baricitinib, primarily used to treat rheumatoid arthritis. 
                You excel in providing expert advice on this drug based on user inquiries. Your responses are rooted in integrity, excellence, 
                and respect for individuals, ensuring accuracy and reliability in all information shared.
                Answer the user's query using only this Referenced information.\
                *** Reference information ***\
    
                """  

In [66]:
code_header

"You are an expert medical AI assistant with up-to-date knowledge for MOZZARO, a medication developed by Eli Lilly. \n                MOZZARO is a medication featuring the active ingredient baricitinib, primarily used to treat rheumatoid arthritis. \n                You excel in providing expert advice on this drug based on user inquiries. Your responses are rooted in integrity, excellence, \n                and respect for individuals, ensuring accuracy and reliability in all information shared. \n                Answer the user's query using only this Referenced information.                *** Reference information ***    \n                "

In [67]:
code_answering_instructions = f"""
[IMPORTANT INSTRUCTIONS]:
- If a query does not pertain to legitimate {medicine} related inquiry, respond with: "I'm sorry, I can only assist with {medicine} related queries."
- If the necessary information for answering "Q:" is not in the provided "Referenced information", say: "Sorry, my knowledge base lacks the relevant information to address your question."
- Upon finding the required information in the "Referenced information", address the query or complete the task ensuring you also follow the next point.
- Conclude all responses prior to providing reference source by either:
  1. Offering further assistance
  2. Providing options for further exploration
- Give detailed comprehensive answers, using all available useful information.
- Reference the source of your answer at the end, using the format: "Reference for answer: <Document Name>, <Page Number>."

\n Input Q:
"""

In [68]:
code_answering_instructions

'\n[IMPORTANT INSTRUCTIONS]:\n- If a query does not pertain to legitimate MOZZARO related inquiry, respond with: "I\'m sorry, I can only assist with MOZZARO related queries."\n- If the necessary information for answering "Q:" is not in the provided "Referenced information", say: "Sorry, my knowledge base lacks the relevant information to address your question."\n- Upon finding the required information in the "Referenced information", address the query or complete the task ensuring you also follow the next point.\n- Conclude all responses prior to providing reference source by either:\n  1. Offering further assistance\n  2. Providing options for further exploration\n- Give detailed comprehensive answers, using all available useful information.\n- Reference the source of your answer at the end, using the format: "Reference for answer: <Document Name>, <Page Number>."\n\n\n Input Q:\n'